In [ ]:
import hpc05
import matplotlib.pyplot as plt
import scipy.optimize

In [ ]:
hpc05.kill_remote_ipcluster()

Start up cluster we use the `hpc05` [package](https://github.com/basnijholt/hpc05), use your favorite method to get an `ipyparallel.client.view.LoadBalancedView` object.

In [ ]:
client, dview, lview = hpc05.start_remote_and_connect(100, profile='pbs', timeout=300,
                                                      env_path='~/.conda/envs/kwant_dev/',
                                                      folder='~/disorder_invariants/code/',
                                                     )

In [ ]:
%%px --local
import itertools as it
import scipy
import scipy.linalg as la
import numpy as np
import copy
import functools as ft

import kwant
# print(kwant.__version__)
from lcao import L_matrices, lcao_term
from hamiltonians import SnTe_6band_disorder #, doped_m
from hamiltonians import SnTe_6band_params as SnTe_params

In [ ]:
%%px --local
# Lead parameters to make sure it doesn't have zero velocity modes
parsl=dict(ml=0, tl=np.array([2, 0.5, 0.5]), k_x=0, k_y=0)

# Redefine `doped_m` to work here
def doped_m(site, doping, mSn, mPb, mTe, rng):
    tag = site.tag
    a = np.sum(tag) % 2
    if a == 0:
        return mTe
    else:
        ran = rng.uniform()
        if ran < doping:
            return mPb
        else:
            return mSn

def SnTe_cylinder(Lx, Ly, Lz=None, with_lead=True):
    if Lz is None:
        Lz = Ly

    syst = SnTe_6band_disorder()

    # Scattering region
    if with_lead:
        builder = kwant.Builder(kwant.lattice.TranslationalSymmetry(Ly * np.array([0, 1, 0]),
                                                                    Lz * np.array([0, 0, 1])))
        builder.fill(syst, lambda site: np.abs(site.pos[0]) <= Lx/2, start=np.zeros(3));
    else:
        builder = kwant.Builder(kwant.lattice.TranslationalSymmetry(Lx * np.array([1, 0, 0]),
                                                                    Ly * np.array([0, 1, 0]),
                                                                    Lz * np.array([0, 0, 1])))
        builder.fill(syst, lambda site: True, start=np.zeros(3));

    builder = kwant.wraparound.wraparound(builder)

    # Make lead simple cubic
    if with_lead:
        systl = kwant.Builder(symmetry=kwant.lattice.TranslationalSymmetry(*np.eye(3)))
        lat = kwant.lattice.cubic(norbs=6)
        systl[lat(0, 0, 0)] = lambda site, ml: ml * np.eye(6)
        systl[kwant.builder.HoppingKind([1, 0, 0], lat)] = lambda site1, site2, tl: tl[0] * np.eye(6)
        systl[kwant.builder.HoppingKind([0, 1, 0], lat)] = lambda site1, site2, tl: tl[1] * np.eye(6)
        systl[kwant.builder.HoppingKind([0, 0, 1], lat)] = lambda site1, site2, tl: tl[2] * np.eye(6)
        lead = kwant.Builder(kwant.TranslationalSymmetry(Ly * np.array([0, 1, 0]),
                                                         Lz * np.array([0, 0, 1]),
                                                         np.array([-1, 0, 0])))
        lead.fill(systl, lambda site: True, start=np.zeros(3));
        lead = kwant.wraparound.wraparound(lead, keep=2)
        builder.attach_lead(lead)
        builder.attach_lead(lead.reversed())
    
    return builder.finalized()

In [ ]:
%%px --local
def localization_length_from_transmissions(lengths, avg_log_transmissions):
    try:
        popt, pcov = scipy.optimize.curve_fit(affine, lengths, avg_log_transmissions)
    except RuntimeError:
        return np.nan, np.nan

    # Uncertainty in the fit slope, allow 1 stddev deviation
    evals, evecs = np.linalg.eig(pcov)
    u_localization_length = np.abs(np.sqrt(evals) @ evecs[0])

    # Use large value for better plotting, could also use np.inf
    large = 1.0e6
    if popt[0] < 0:
        localization_length = -1/popt[0]
    elif abs(popt[0]) < u_localization_length:
        localization_length = large
    else:
        localization_length = large
        print("Warning, poor localization length fit, \n"
              "popt = {}, pcov = {}".format(popt, pcov))
    
    return localization_length, u_localization_length

def affine(x, a, b):
    return a * x + b

In [ ]:
%%px --local
num_m = 51
m_array = np.linspace(-1, 4, num_m)

num_x = 21
x_array = np.linspace(0, 1, num_x)

Ls = np.array([10, 15, 20])
n_dis = 10

W = 12

def job(doping, mPb):
    print(doping, mPb)
    pars = SnTe_params.copy()
    mSn = SnTe_params['m'][1]
    mTe = SnTe_params['m'][0]
    rng = kwant._common.ensure_rng(rng=0)

    pars['m'] = ft.partial(doped_m, doping=doping, mSn=mSn, mPb=mPb, mTe=mTe, rng=rng)
    # gap should be near the weighted average
    pars['mu'] = -(((1 - doping) * mSn + doping * mPb) + mTe) / 2
    pars['k_x'] = pars['k_y'] = pars['k_z'] = 0
    pars['doping'] = doping
    pars['mPb'] = mPb

    # Find gap
    spectrum = kwant.kpm.SpectralDensity(SnTe_cylinder(max(Ls), max(Ls), with_lead=False), 
                                         num_moments=1000, params=pars)
    es, dos = spectrum()
    ran = np.logical_and(-1 < es, es < 1)
    minimum = np.argmin(dos[ran])
    mine = es[ran][minimum]
    mindos = dos[ran][minimum]
    filling = spectrum.integrate(distribution_function=lambda x: x<mine) / spectrum.integrate()

    Ts = []
    var_Ts = []
    for L in Ls:
        syst = SnTe_cylinder(L, W, with_lead=True)
        T, var_T = disorder_averaged_log_transmission(syst, 
                                                     {**pars, **parsl}, 
                                                     n_dis, 
                                                     energy=mine,
                                                     return_var=True)
        Ts.append(T - 2 * np.log(W))
        var_Ts.append(var_T)
    # xi = localization_length_from_transmissions(Ls, Ts)
    # return xi
    return Ts, var_Ts, mine, filling

def disorder_averaged_log_transmission(system, params,
                                       disorder_realizations, energy=0, return_var=False):
    log_transmissions = [np.log(kwant.smatrix(system, energy, params=params).transmission(1, 0))
                         for i in range(disorder_realizations)]
    if return_var:
        return np.mean(log_transmissions, axis=0), np.var(log_transmissions, axis=0)
    else:
        return np.mean(log_transmissions, axis=0)

In [ ]:
mws = [(x, m) for x, m in it.product(x_array, m_array)]
result = lview.map_async(job, *zip(*mws))

In [ ]:
result.wait_interactive()

In [ ]:
all([err is None for err in result.error])

In [ ]:
Ts_vs_xm = np.array(result.get())
xi_vs_xm = np.array([localization_length_from_transmissions(Ls, Ts) for Ts, *_ in Ts_vs_xm])
xi_arr = xi_vs_xm[:, 0]
xi_arr = xi_arr.reshape((len(x_array), len(m_array)))

In [ ]:
plt.imshow(xi_arr.T, 
           vmin=0, vmax=50,
           extent=(x_array[0], x_array[-1], m_array[0], m_array[-1]),
           aspect=0.4, 
           origin='lower')
plt.colorbar()
plt.xlabel('x')
plt.ylabel(r'$m_{Pb}$')
plt.title(r'$\xi$ at fixed W=12 with L up to 20')

In [ ]:
import pickle
pickle.dump(dict(params=SnTe_params, Ls=Ls, E=0, W=10,
                 m_array=m_array, x_array=x_array, 
                 Ts=Ts_vs_xm, xi_array=xi_arr,
                 disorder_realizations=n_dis),
            open('../data/Localization_SnXTe_6orb.pickle', 'wb'))